In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt




In [ ]:
# Include all packages
import os
import shutil
import cv2
from time import time
from datetime import datetime
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from yolov5.models.yolo import Model
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import torchvision
from torchvision import transforms

from yolov5.utils.datasets import LoadImagesAndLabels
from yolov5.utils.general import check_requirements


In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/DataSet.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
except:
    print("Using Local Machine")

In [ ]:
def ResizeImage(image: np.ndarray, x1: int, y1: int, x2: int, y2: int, newWidth: int, newHeight: int) -> tuple:
    originalHeight, originalWidth = image.shape[:2]
    # widthScale = newWidth / originalWidth
    # heightScale = newHeight / originalHeight
    # resizedImage = cv2.resize(
    #     image, (newWidth, newHeight), interpolation=cv2.INTER_LINEAR)
    # x1New, y1New = int(x1 * widthScale), int(y1 * heightScale)
    # x2New, y2New = int(x2 * widthScale), int(y2 * heightScale)
    scale = min(newWidth / originalWidth, newHeight / originalHeight)
    resizedImage = cv2.resize(image, (round(originalWidth * scale), round(originalHeight * scale)), interpolation=cv2.INTER_LINEAR)
    dx = round((newWidth - resizedImage.shape[1]) / 2)
    dy = round((newHeight - resizedImage.shape[0]) / 2)
    paddedImage = cv2.copyMakeBorder(resizedImage, dy, dy, dx, dx, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    x1New, y1New = int(x1 * scale + dx), int(y1 * scale + dy)
    x2New, y2New = int(x2 * scale + dx), int(y2 * scale + dy)
    return paddedImage, x1New, y1New, x2New, y2New


In [ ]:
def LoadDataSet(dataSetFolderPath: str) -> tuple:
    images = []
    annotations = []
    annotationsFilePath = dataSetFolderPath+"/allAnnotations.csv"
    annotationsDataFrame = pd.read_csv(annotationsFilePath, sep=";")
    uniqueSigns = annotationsDataFrame['Annotation tag'].unique().tolist()
    for index, row in annotationsDataFrame[1:].iterrows():
        image = cv2.imread(dataSetFolderPath+"/"+row[0])
        images.append(image)
        annotations.append(
            [row[2], row[3], row[4], row[5]])

    del annotationsDataFrame

    return images, annotations, len(uniqueSigns)


In [ ]:
def PreProcessDataSet(images: list, annotations: list, batchSize: int, resize: tuple) -> tuple:
    resizedImages = []
    newAnnotations = []
    for i, image in enumerate(images):
        [x1, y1, x2, y2] = annotations[i]
        resizedImage, x1New, y1New, x2New, y2New = ResizeImage(
            image, x1, y1, x2, y2, resize[0], resize[1])
        resizedImages.append(resizedImage)
        newAnnotations.append(
            [x1New, y1New, x2New, y2New])

    X_train, X_val, y_train, y_val = train_test_split(
        resizedImages, newAnnotations, test_size=0.3, random_state=42)

    return X_train, X_val, y_train, y_val


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        inputData, label = self.data[idx]

        if self.transform:
            inputData = self.transform(inputData)
        inputData = torch.from_numpy(inputData).float()
        label = torch.tensor(label).float()
        return inputData, label

def CreateDataLoaders(X_train, X_val, y_train, y_val, batchSize):
    trainDataSet = []
    valDataSet = []
    for i in range(len(X_train)):
        trainDataSet.append((X_train[i], y_train[i]))

    for i in range(len(X_val)):
        valDataSet.append((X_val[i], y_val[i]))

    trainDataSet = CustomDataset(trainDataSet)
    valDataSet = CustomDataset(valDataSet)
    trainDataLoader = DataLoader(
        trainDataSet, batch_size=batchSize, shuffle=True, num_workers=4)
    valDataLoader = DataLoader(
        valDataSet, batch_size=batchSize, shuffle=False, num_workers=4)

    return trainDataLoader, valDataLoader


In [ ]:
def TargetstoTensors(targets, batch_size, numAnchors, gridSizes, numClasses):
    targetObj = []
    targetClass = []
    targetBox = []
    # batch_size = 32
    for grid_size in gridSizes:
        targetObj.append(torch.zeros(
            (batch_size, numAnchors, grid_size, grid_size, 1)))
        targetClass.append(torch.zeros(
            (batch_size, numAnchors, grid_size, grid_size, numClasses)))
        targetBox.append(torch.zeros(
            (batch_size, numAnchors, grid_size, grid_size, 4)))
    # current_batch_size = int(targets[:, 0].max()) + 1
    # print("current_batch_size",targets.size(0))
    for target in targets:
        batchIndex, cls, xCenter, yCenter, width, height = target.long()

        for i, grid_size in enumerate(gridSizes):

            x_cell, y_cell = int(
                xCenter * grid_size), int(yCenter * grid_size)
            anchor = 0
            try:
                targetObj[i][batchIndex, anchor, y_cell, x_cell, 0] = 1
                targetClass[i][batchIndex, anchor, y_cell, x_cell, cls] = 1
                targetBox[i][batchIndex, anchor, y_cell, x_cell] = torch.tensor(
                    [xCenter, yCenter, width, height])
            except Exception as e:
                # print(e)
                pass
    return targetObj, targetClass, targetBox


In [ ]:
class YOLOv5Loss(nn.Module):
    def __init__(self, numClasses, numAnchors=3):
        super(YOLOv5Loss, self).__init__()
        self.numClasses = numClasses
        self.numAnchors = numAnchors

    def forward(self, preds, targets):
        objectLoss = torch.tensor(0.0, device=preds[0].device)
        classLoss = torch.tensor(0.0, device=preds[0].device)
        boxLoss = torch.tensor(0.0, device=preds[0].device)
        batch_size = preds[0].size(0)
        gridSizes = [pred.size(2) for pred in preds]
        targetObjList, targetClassList, targetBoxList = TargetstoTensors(
            targets, batch_size, self.numAnchors, gridSizes, self.numClasses)
        for i, pred in enumerate(preds):
            # pred_obj = pred[..., 4].sigmoid()
            # pred_cls = pred[..., 5:].sigmoid()
            # pred_box = pred[..., :4]

            targetObj = targetObjList[i].to(pred.device)
            targetClass = targetClassList[i].to(pred.device)
            targetBox = targetBoxList[i].to(pred.device)

            objectLoss += nn.BCEWithLogitsLoss()(pred[..., 4:5], targetObj)
            classLoss += nn.BCEWithLogitsLoss()(pred[..., 5:], targetClass)
            boxLoss += nn.MSELoss()(pred[..., :4], targetBox)

        totalLoss = objectLoss + classLoss + boxLoss
        return totalLoss


In [ ]:
def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    model.load_state_dict(torch.load("yolov5{}.pt".format(version))["model"].state_dict(), strict=False)

    return model


In [ ]:
def TrainModel(model, dataLoader, epochs, optimizer, lossFunction, device):
    model.train()
    for epoch in range(epochs):
        print("Epoch {}/{}:".format(epoch+1, epochs))
        startTime = time()
        runningLoss = 0
        dataLoaderLen = len(dataLoader)
        for i, (inputs, targets) in enumerate(dataLoader):
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs.to(device)
            targets = targets.to(device)
            optimizer.zero_grad()
            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                loss = lossFunction(outputs, targets)
                loss.backward()
                optimizer.step()

            runningLoss += loss.item() * inputs.size(0)
            if(((i*100)//dataLoaderLen) % 10 == 0):
                print((i*100//dataLoaderLen), end="%,")
        endTime = time()
        timeTaken = endTime-startTime
        epochLoss = runningLoss / dataLoaderLen
        print()
        print("Training Loss: {:.4f}".format(epochLoss))
        print("Time taken: {}min, {}, secs".format(timeTaken//60, timeTaken % 60))

    print("Training complete.")
    return model


In [ ]:
def EvaluateModel(model, dataLoader, device,  numClasses, conf_thres=0.5, iou_thres=0.5):
    print("Evaluateing Model:")
    model.eval()
    detections = []
    labels = []
    startTime = time()
    dataLoaderLen = len(dataLoader)
    conf_thres = torch.tensor(conf_thres)
    conf_thres = conf_thres.to(device)
    
    with torch.no_grad():
        for inputs, label in dataLoader:
            inputs = inputs.permute(0, 3, 1, 2)
            inputs = inputs.to(device)
            label = label.to(device)
            
            output = model(inputs)
            output = torchvision.ops.nms(output[0], conf_thres, iou_thres)
            
            detections.extend(output.cpu().numpy())
            labels.extend(label.cpu().numpy())

    coco_gt = COCO()
    coco_gt.dataset['annotations'] = labels
    coco_gt.createIndex()

    coco_dt = coco_gt.loadRes(detections)
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.imgIds = list(range(len(labels)))
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats


In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 100
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("Using {} device".format(device))

In [ ]:
print("Downloading Weights of yolo5 Verion ", yolo5Version)
weightsURL = f"https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(yolo5Version)
!wget {weightsURL}

In [ ]:
images, annotations, numClasses = LoadDataSet("./DataSet")


In [ ]:
X_train, X_val, y_train, y_val = PreProcessDataSet(
    images, annotations, batchSize, inputShape)
del images
del annotations


In [ ]:
trainDataLoader, valDataLoader = CreateDataLoaders(
    X_train, X_val, y_train, y_val, batchSize)
del X_train
del y_train
del X_val
del y_val


In [ ]:
yolov5Model = CreateYolov5Model(numClasses,yolo5Version)
optimizer = optim.Adam(yolov5Model.parameters(), lr=0.001)
yolov5LossFunction= YOLOv5Loss(numClasses=numClasses)
yolov5Model = yolov5Model.to(device)
yolov5LossFunction = yolov5LossFunction.to(device)

In [ ]:
trainedModel = TrainModel(yolov5Model, trainDataLoader, epochs, optimizer, yolov5LossFunction, device)

In [ ]:
date = datetime.now()
date = date.strftime("%m-%d-%H")
torch.save(trainedModel.state_dict(), 'yolov5Model' + date +'.pth')
shutil.copy('/content/yolov5Model' + date +'.pth', '/content/drive/MyDrive/DL Project/Trained Models/yolov5Model' + date +'.pth')


In [ ]:
evaluationMetrics = EvaluateModel(yolov5Model, valDataLoader, device, numClasses)


In [ ]:
print("COCO Evaluation Metrics:")
print(" Average Precision (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {:.3f}".format(evaluation_metrics[0]))
print(" Average Precision (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = {:.3f}".format(evaluation_metrics[1]))
print(" Average Precision (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = {:.3f}".format(evaluation_metrics[2]))
print(" Average Precision (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {:.3f}".format(evaluation_metrics[3]))
print(" Average Precision (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {:.3f}".format(evaluation_metrics[4]))
print(" Average Precision (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {:.3f}".format(evaluation_metrics[5]))
print(" Average Recall (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = {:.3f}".format(evaluation_metrics[6]))
print(" Average Recall (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = {:.3f}".format(evaluation_metrics[7]))
print(" Average Recall (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = {:.3f}".format(evaluation_metrics[8]))
print(" Average Recall (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = {:.3f}".format(evaluation_metrics[9]))
print(" Average Recall (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = {:.3f}".format(evaluation_metrics[10]))
print(" Average Recall (AR) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = {:.3f}".format(evaluation_metrics[11]))
